# Investigating Methodologies for Increasing the Effectiveness of Transportation Networks

## Complexity Science Project 1
### Jonathan Zerez, Nathan Lepore

First, load the data from Open Street View into OSMNX DiGraph Objects. Additionally, recast the length fields to be floats instead of strings. The possible cities to view are ** Amsterdam, Detroit, Copenhagen, and Manhattan. **

In [12]:
import numpy as np
import networkx as nx
import os
import osmnx as ox
import time
import matplotlib.pyplot as plt

def convert_lengths_to_floats(G):
    """
    Changes the attributes of the edges of graph G such that the 'length' field
    is a float rather than a string
    """
    for u, v, d in list(G.edges(data=True)):
        d['length'] = float(d['length'])


# Call this function to get the data 
def get_data(city='Detroit', mode='bike'):
    '''
    Loads city transportation network data for a specified mode of transportation
    city: The string of the city to use. Included in this project are Detroit, Amsterdam, Copenhagen, and Manhattan
    mode: The mode of transportation to import.
    '''
    G = ox.load_graphml('{}_{}.graphml'.format(city, mode), folder='../data/{}/'.format(city))
    convert_lengths_to_floats(G)
    return G


#### Define helper functions to compute metrics

In [7]:
def calc_directness(G, u, v):
    """
        Calculate the directness between two nodes u and v
        Directness is the ratio between:
            - the shortest node path length (number of edges times the length of each edge)
            - the straight line distance between those two nodes
        If v cannot be reached from u, then directness is zero
        
        G: An nx graph
        u: the starting node
        v: the ending node
        
    """
    try:
        path_length = nx.shortest_path_length(G, source=u, target=v, weight='length')
        direct_length = new_edge_length(G, u, v)
        ret = direct_length/path_length
    except:
        return 0
    return ret


def calc_avg_directness(G, pairs):
    """
    Calculates the average directness of n pairs of nodes. All of these nodes are already connected.
    G: An nx graph
    n: The number of pairs to use
    """
    d = 0
    n = len(pairs)
    for pair in pairs:
        d += calc_directness(G, pair[0], pair[1]) / n

    return d
        

def get_components(G):
    '''
    Get the connected components of G
    G: An nx graph
    
    Outputs
    wcc: A list of the components sorted by size
    '''
    wcc = [cc for cc in nx.weakly_connected_component_subgraphs(G)]
    wcc.sort(key=len, reverse=True)
    return wcc

def calc_lcc(G):
    '''
    Returns the number of nodes in the largest component of graph G
    G: An nx graph
    '''
    wcc = get_components(G)
    return len(wcc[0])

def euclidean_dist_vec(y1, x1, y2, x2):
    '''
    Calculate the euclidean distance between two points.
    '''
    distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
    return distance

def connectedness(G):
    N = len(G)
    wcc = get_components(G)
    largest = wcc[0]
    N_ = len(largest)
    return N_/N

def make_random_pairs(G_bike, G_car, n = 250):
    '''
    Creates n pairs of random nodes from graph G
    G: an nx graph
    n: the number of pairs to return
    '''
    bike_pairs = []
    car_pairs = []
    for _ in range(n):
        bike_pair = np.random.choice(G_bike.nodes(), 2)
        b1 = G_bike.nodes[bike_pair[0]]
        b2 = G_bike.nodes[bike_pair[1]]
        bike_pairs.append(bike_pair)
                

        u = ox.get_nearest_node(G_car, (b1['x'], b1['y']))
        v = ox.get_nearest_node(G_car, (b2['x'], b2['y']))
        car_pairs.append((u,v))
    return bike_pairs, car_pairs

#### Define Methods for connecting the Graphs

In [8]:
def L2S(wcc):
    '''
    Largest to Second Algorithm
    Find the closest pair of nodes between the largest component of the graph, and the second largest component
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for i in wcc[0].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for j in wcc[1].nodes(data=True):
            j_coord = (j[1]['y'], j[1]['x'])
            dist = euclidean_dist_vec(i_coord[0], i_coord[1], j_coord[0], j_coord[1])
            if dist < closest_pair['dist']:
                closest_pair['i'] = i[0]
                closest_pair['j'] = j[0]
                closest_pair['dist'] = dist
    return closest_pair

def L2C(wcc):
    '''
    Largest to Closest Algorithm
    Find the closest pair of nodes between the largest component of the graph, and the closest other component
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for i in wcc[0].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for j in wcc[1:]:
            for k in j.nodes(data=True):
                j_coord = (k[1]['y'], k[1]['x'])
                dist = euclidean_dist_vec(float(i_coord[0]), float(i_coord[1]), float(j_coord[0]), float(j_coord[1]))
                if dist < closest_pair['dist']:
                    closest_pair['i'] = i[0]
                    closest_pair['j'] = k[0]
                    closest_pair['dist'] = dist
    return closest_pair

def R2C(wcc):
    '''
    Random to Closest Algorithm
    Find the closest pair of nodes between two a random component of the graph, and the closest other component
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    num_clusters = len(wcc)
    cluster = np.random.choice(len(wcc))
    for i in wcc[cluster].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for w,j in enumerate(wcc[1:]):
            if w == cluster - 1:
                break
            for k in j.nodes(data=True):
                j_coord = (k[1]['y'], k[1]['x'])
                dist = euclidean_dist_vec(float(i_coord[0]), float(i_coord[1]), float(j_coord[0]), float(j_coord[1]))
                if dist < closest_pair['dist']:
                    closest_pair['i'] = i[0]
                    closest_pair['j'] = k[0]
                    closest_pair['dist'] = dist
    return closest_pair

def Closest(wcc):
    '''
    Closest Algorithm
    Find the closest pair of nodes between the two closest components in the graph
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for v,i in enumerate(wcc[0:]):
            for u in i.nodes(data=True):
                i_coord = (u[1]['y'], u[1]['x'])
                for w,j in enumerate(wcc[0:]):
                    if w == v:
                        break
                    for k in j.nodes(data=True):
                        j_coord = (k[1]['y'], k[1]['x'])
                        dist = euclidean_dist_vec(float(i_coord[0]), float(i_coord[1]), float(j_coord[0]), float(j_coord[1]))
                        if dist < closest_pair['dist']:
                            closest_pair['i'] = u[0]
                            closest_pair['j'] = k[0]
                            closest_pair['dist'] = dist
    return closest_pair


### Define a function to apply an algorithm to a city, and return the computed results for each edge added to the graph

In [9]:
def new_city(G, algy, length, random_pairs, debug=False):
    """
    G = an NX Digraph
    algy = desired path-adding algorithm
    length = total length of bike path to add (meters)
    random_pairs = A list of random pairs of nodes used to caclulate directness
    
    """
    added_edges = []
    tot_length = [0]
    directness_random = []
    lccs = []
    connectedness_rank = []

    while sum(tot_length) < length:
        wcc = get_components(G)
        if debug:
            print(sum(tot_length) / length, '% Complete')
        lccs.append(len(wcc[0]))
        directness_random.append(calc_avg_directness(G, random_pairs))
        
        connected_rank = connectedness_rank.append(connectedness(G))
        added_edge = algy(wcc)
        edge = (added_edge['i'],added_edge['j'],added_edge['dist'])
        G.add_edge(edge[0], edge[1], length = edge[2])
        added_edges.append(edge)
        tot_length.append(edge[2])
        
    wcc = get_components(G)
    lccs.append(len(wcc[0]))
    directness_random.append(calc_avg_directness(G, random_pairs))

    connected_rank = connectedness_rank.append(connectedness(G))
    return G, added_edges, tot_length, directness_random, lccs, connectedness_rank

### Run the study!
We will start wtih Detroit and the L2S algorithm

In [10]:
name = 'Detroit'
G_bikes = get_data(name)
G_cars = get_data(name, mode='drive')

bike_pairs, car_pairs = make_random_pairs(G_bikes, G_cars, n=500)
calc_avg_directness(G_bikes, bike_pairs)
G_new, edges, lengths, directness, lccs, connectedness_rank = new_city(G_bikes, L2S, 30000, bike_pairs)

0.0

### Plot the results

In [6]:
plt.plot(np.cumsum(lengths), directness - directness[0])
plt.plot(np.cumsum(lengths), lccs)
plt.show()

NameError: name 'lengths' is not defined

### Define a function to determine whether a car or a bike is taken for a certain route

In [13]:
def route_decision(directness_bike, directness_car, threshold):
    '''
    directness_bike: The directness of taking a bike between two different nodes
    directness_car: The directness of taking a car between two different nodes
    threshold: decision threshold (from zero to one)
        if threshold == 1: the user will always choose to bike
        if threshold == 0.5: the user will always choose the more efficient route
        if threshold == 0: the user will always choose to drive
    
    returns 1 if the user bikes
    returns 0 if the user drives
    '''
    dcar = directness_car**-1
    dbike = directness_bike**-1
    
    return (1-threshold)*dbike < (threshold)*dcar
        

In [ ]:
# Average directness for the car layer
rcc = calc_avg_directness(G_cars, car_pairs) 
# Number of threshold values to test
nthresh = 101
thresholds = np.linspace(0, 1, nthresh)
ds = np.empty((nthresh, len(directness)))

# Iterate over the possible directness and threshold values
for i in range(len(directness)):
    for j in range(nthresh):
        bcc = directness[i]
        ds[j][i] = route_decision(bcc, rcc, thresholds[j])

# plot data
plt.imshow(ds, interpolation='nearest', aspect='auto')
plt.xticks(np.arange(0, len(lengths), step = 3), np.round(np.cumsum(lengths)[::3], decimals=3), rotation=45)
plt.yticks(np.arange(0, nthresh, step=10), np.round(thresholds[::10], decimals=1))
plt.colorbar()

plt.xlabel('Length of Road Added (m)')
plt.ylabel('Threshold')
plt.title('Driving vs. Biking Threshold vs Length of Road Added')
plt.show()